In [ ]:
## Libraries Required
!pip install langchain-huggingface
## For API Calls
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain
!pip install pypdf
!pip install sentence-transformers
!pip install langchain-community
!pip install langchain chromadb transformers sentence-transformers
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
!pip install langchain faiss-cpu transformers sentence-transformers
from langchain.vectorstores import FAISS
!pip install langchain faiss-cpu transformers sentence-transformers accelerate
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
!pip install langchain faiss-cpu transformers sentence-transformers cachetools
import cachetools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
from huggingface_hub import login
sec_key="your token"
# Use your token here
login(token="your token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os

# Set your Hugging Face token here
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'insert your token'


In [ ]:
from google.colab import files

# This will prompt you to upload a file
uploaded = files.upload()

Saving test.pdf to test.pdf


In [ ]:
file_name = list(uploaded.keys())[0]

In [ ]:
print(f"File uploaded and saved as: {file_name}")

File uploaded and saved as: test.pdf


In [ ]:
file_path = f"/content/{file_name}"

print(f"The file path is: {file_path}")


The file path is: /content/test.pdf


# Starting with RAG

In [ ]:

# Step 1: Load and Process PDF
pdf_path = "/content/test.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split the text for vectorization
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Convert text to vectors using an appropriate Hugging Face model
# Use the model name directly, not as a `repo_id`
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(texts, embedding=embeddings)

# Step 4: Set up RAG with LangChain
# Define the LLM model (replace with an appropriate model ID)
llm = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.7})  # Example LLM

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")



<ipython-input-9-b44d5e15d8ec>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-b44d5e15d8ec>:17: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  llm = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.7})  # Example LLM


In [ ]:
#normal llm
import time
start_time = time.time()
normal_response = llm.invoke("What is omniverse?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print("Time taken for normal LLM:", normal_time)

# RAG LLM Output
start_time = time.time()
rag_response = rag_chain.run("What is omniverse?")
rag_time = time.time() - start_time
print("RAG LLM Output:", rag_response)

print("Time taken for RAG LLM:", rag_time)


# Compare the time difference
print("Time difference between Normal and RAG:", rag_time - normal_time)

Normal LLM Output: What is omniverse?

A universal language: a language of order.

Linguistic order: the form of order that describes a set of physical properties.

The order of a set of physical properties: a set of propositional rules or rules that describe orders that are given for a given set of physical properties.

A set of physical rules or rules that describe orders that are given for a given set of physical properties. An order: the particular form of order that describes a set of physical
Time taken for normal LLM: 0.07065010070800781
RAG LLM Output: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

reducing latency and increasing performance efficiency.  
3. NVIDIA Omniverse  
NVIDIA Omniverse is a platform designed for creating and simulating complex virtual worlds. It 
integrates AI, simulation, and real -time graphics to enable collaborative 3D design and 
sim

In [ ]:
#normal llm
import time
start_time = time.time()
normal_response = llm.invoke("What are surface features of moon?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print()
print()
print("Time taken for normal LLM:", normal_time)
print()
print()

# RAG LLM Output
start_time = time.time()
rag_response = rag_chain.run("What are surface features of moon?")
rag_time = time.time() - start_time

print("RAG LLM Output:", rag_response)
print()
print()
print("Time taken for RAG LLM:", rag_time)
print()
print()
print()
print()

# Compare the time difference
print("Time difference between Normal and RAG:", rag_time - normal_time)

Normal LLM Output: What are surface features of moon? Should you see the features first?

Moon is one of the most beautiful objects in the universe and we are lucky enough to have seen it before. It is the largest body in the universe with a diameter of 2.29 light-years (light years). It is round, with a diameter of 1.38 light-years.

The moon is filled with a slightly different type of material called Titanite. It is composed of a mixture of Earth's atmosphere and Titanite. This


Time taken for normal LLM: 0.0711507797241211


RAG LLM Output: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(238,855 miles) from Earth. The Moon’s gravity affects Earth’s t ides and has a significant impact 
on life.  
Surface Features  
The Moon’s surface is marked by a variety of features:  
1. Maria: These are large, dark plains formed by ancient volcanic eruptions. They make up 
about 1

FILE HANDLING

In [ ]:
import os

# List all files in the current directory
for filename in os.listdir('/content'):
    print(filename)


.config
test.pdf
sample_data


In [ ]:
os.remove('/content/modi.pdf')

## Optimization STARTS: 1st Technique - Quantization

In [ ]:


# Step 1: Load and Process PDF
pdf_path = "/content/test.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split the text for vectorization
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Convert text to vectors using an appropriate Hugging Face model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(texts, embedding=embeddings)

# Step 4: Quantize the GPT-2 model
quantization_config = {
    'load_in_8bit': True  # Enable 8-bit quantization
}

# Load the quantized GPT-2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2", **quantization_config)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Create the HuggingFaceHub instance with quantized model
from langchain.llms import HuggingFaceHub

llm_quantized = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={"temperature": 0.7}
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
rag_chain = RetrievalQA.from_chain_type(llm=llm_quantized, retriever=retriever, chain_type="stuff")

# Step 5: Normal LLM Output (without RAG)
import time
start_time = time.time()
normal_response = llm_quantized.invoke("What is omniverse?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print("Time taken for normal LLM:", normal_time)

# Step 6: RAG LLM Output (with RAG and quantization)
start_time = time.time()
rag_response = rag_chain.run("What is omniverse?")
rag_time = time.time() - start_time

print("RAG LLM Output:", rag_response)
print("Time taken for RAG LLM:", rag_time)

# Step 7: Compare the time difference and calculate percentage reduction
time_reduction_percent = ((normal_time - rag_time) / normal_time) * 100
print("Time difference between Normal and RAG:", rag_time - normal_time)
print("Time reduction percentage due to quantization:", time_reduction_percent)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Normal LLM Output: What is omniverse?

A universal language: a language of order.

Linguistic order: the form of order that describes a set of physical properties.

The order of a set of physical properties: a set of propositional rules or rules that describe orders that are given for a given set of physical properties.

A set of physical rules or rules that describe orders that are given for a given set of physical properties. An order: the particular form of order that describes a set of physical
Time taken for normal LLM: 0.06734490394592285
RAG LLM Output: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

reducing latency and increasing performance efficiency.  
3. NVIDIA Omniverse  
NVIDIA Omniverse is a platform designed for creating and simulating complex virtual worlds. It 
integrates AI, simulation, and real -time graphics to enable collaborative 3D design and 
sim

## Optimization STARTS: 2st Technique - Using FAISS

In [ ]:

# Step 1: Load and Process PDF
pdf_path = "/content/test.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split the text for vectorization
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Convert text to vectors using an appropriate Hugging Face model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding=embeddings)

# Step 4: Set up RAG with LangChain using HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={"temperature": 0.7}
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# Step 5: Normal LLM Output (without RAG)
import time
start_time = time.time()
normal_response = llm.invoke("What is omniverse?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print("Time taken for normal LLM:", normal_time)

# Step 6: RAG LLM Output (with RAG using FAISS)
start_time = time.time()
rag_response = rag_chain.run("What is omniverse?")
rag_time = time.time() - start_time

print("RAG LLM Output with FAISS:", rag_response)
print("Time taken for RAG LLM with FAISS:", rag_time)

# Step 7: Compare the time difference and calculate percentage reduction
time_reduction_percent = ((normal_time - rag_time) / normal_time) * 100
print("Time difference between Normal and RAG:", rag_time - normal_time)
print("Time reduction percentage due to FAISS:", time_reduction_percent)


Normal LLM Output: What is omniverse?

A universal language: a language of order.

Linguistic order: the form of order that describes a set of physical properties.

The order of a set of physical properties: a set of propositional rules or rules that describe orders that are given for a given set of physical properties.

A set of physical rules or rules that describe orders that are given for a given set of physical properties. An order: the particular form of order that describes a set of physical
Time taken for normal LLM: 0.07000565528869629
RAG LLM Output with FAISS: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

reducing latency and increasing performance efficiency.  
3. NVIDIA Omniverse  
NVIDIA Omniverse is a platform designed for creating and simulating complex virtual worlds. It 
integrates AI, simulation, and real -time graphics to enable collaborative 3D desi

## Optimization STARTS: 3rd Technique - Using Accelerate for Model Parallelism

In [ ]:

# Initialize Accelerator for model parallelism
accelerator = Accelerator()

# Step 1: Load and Process PDF
pdf_path = "/content/test.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split the text for vectorization
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Convert text to vectors using an appropriate Hugging Face model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding=embeddings)

# Step 4: Load the GPT-2 model with model parallelism
model = AutoModelForCausalLM.from_pretrained("gpt2")
model = accelerator.prepare(model)  # Prepare the model for distributed execution

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Create the HuggingFaceHub instance with model parallelism
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={"temperature": 0.7}
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# Step 5: Normal LLM Output (without RAG)
import time
start_time = time.time()
normal_response = llm.invoke("What is omniverse?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print("Time taken for normal LLM:", normal_time)

# Step 6: RAG LLM Output (with RAG using model parallelism)
start_time = time.time()
rag_response = rag_chain.run("What is omniverse?")
rag_time = time.time() - start_time

print("RAG LLM Output with model parallelism:", rag_response)
print("Time taken for RAG LLM with model parallelism:", rag_time)

# Step 7: Compare the time difference and calculate percentage reduction
time_reduction_percent = ((normal_time - rag_time) / normal_time) * 100
print("Time difference between Normal and RAG:", rag_time - normal_time)
print("Time reduction percentage due to model parallelism:", time_reduction_percent)


Normal LLM Output: What is omniverse?

A universal language: a language of order.

Linguistic order: the form of order that describes a set of physical properties.

The order of a set of physical properties: a set of propositional rules or rules that describe orders that are given for a given set of physical properties.

A set of physical rules or rules that describe orders that are given for a given set of physical properties. An order: the particular form of order that describes a set of physical
Time taken for normal LLM: 0.06744575500488281
RAG LLM Output with model parallelism: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

reducing latency and increasing performance efficiency.  
3. NVIDIA Omniverse  
NVIDIA Omniverse is a platform designed for creating and simulating complex virtual worlds. It 
integrates AI, simulation, and real -time graphics to enable collabora

## Optimization STARTS: 4th Technique - Implement Caching


In [ ]:

# Step 1: Load and Process PDF
pdf_path = "/content/test.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split the text for vectorization
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Convert text to vectors using an appropriate Hugging Face model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding=embeddings)

# Step 4: Set up a simple cache for retrieved documents
cache = cachetools.LRUCache(maxsize=100)  # LRU Cache with a capacity of 100 entries

def get_documents(query):
    # Check if the result is already in the cache
    if query in cache:
        return cache[query]

    # If not, retrieve the documents and store them in the cache
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
    documents = retriever.get_relevant_documents(query)
    cache[query] = documents
    return documents

# Step 5: Set up RAG with LangChain using HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={"temperature": 0.7}
)

# Normal LLM Output (without RAG)
import time
start_time = time.time()
normal_response = llm.invoke("What is omniverse?")
normal_time = time.time() - start_time

print("Normal LLM Output:", normal_response)
print("Time taken for normal LLM:", normal_time)

# RAG LLM Output (with caching)
start_time = time.time()
documents = get_documents("What is omniverse?")
rag_response = llm.invoke("What is omniverse?" + " ".join([doc.page_content for doc in documents]))
rag_time = time.time() - start_time

print("RAG LLM Output with caching:", rag_response)
print("Time taken for RAG LLM with caching:", rag_time)

# Compare the time difference and calculate percentage reduction
time_reduction_percent = ((normal_time - rag_time) / normal_time) * 100
print("Time difference between Normal and RAG:", rag_time - normal_time)
print("Time reduction percentage due to caching:", time_reduction_percent)


Normal LLM Output: What is omniverse?

A universal language: a language of order.

Linguistic order: the form of order that describes a set of physical properties.

The order of a set of physical properties: a set of propositional rules or rules that describe orders that are given for a given set of physical properties.

A set of physical rules or rules that describe orders that are given for a given set of physical properties. An order: the particular form of order that describes a set of physical
Time taken for normal LLM: 0.07202982902526855
RAG LLM Output with caching: What is omniverse?reducing latency and increasing performance efficiency.  
3. NVIDIA Omniverse  
NVIDIA Omniverse is a platform designed for creating and simulating complex virtual worlds. It 
integrates AI, simulation, and real -time graphics to enable collaborative 3D design and 
simulation.  
• Real-Time Ray Tracing : Provides photorealistic rendering with real -time ray tracing 
capabilities, enhancing the visua